# 卷积层代码实现：

---

## 一、手动实现互相关运算（理解原理）

### 代码

```python
import torch

def corr2d(X, K):
    """X是输入，K是卷积核"""
    h, w = K.shape                              # 拿到卷积核的高和宽
    Y = torch.zeros((X.shape[0] - h + 1,        # 输出的高
                      X.shape[1] - w + 1))       # 输出的宽
    for i in range(Y.shape[0]):                  # 遍历输出的每一行
        for j in range(Y.shape[1]):              # 遍历输出的每一列
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()  # 取小块 × 核，求和
    return Y
```

### 逐行翻译

```python
h, w = K.shape
```

```
卷积核K是一个小矩阵，比如2×2
h = 2（高）
w = 2（宽）

这行就是：看看卷积核多大
```

---

```python
Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
```

```
创建一个全是0的输出矩阵

输出的大小怎么算？
    输出高 = 输入高 - 核高 + 1
    输出宽 = 输入宽 - 核宽 + 1

比如输入3×3，核2×2：
    输出高 = 3 - 2 + 1 = 2
    输出宽 = 3 - 2 + 1 = 2
    所以输出是2×2
```

---

```python
for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
```

```
两个循环：遍历输出矩阵的每个位置

输出是2×2的话：
    i=0, j=0 → 左上角
    i=0, j=1 → 右上角
    i=1, j=0 → 左下角
    i=1, j=1 → 右下角
```

---

```python
Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
```

**这是最核心的一行！拆开看：**

```
X[i:i+h, j:j+w]
→ 从输入X中取出一个和卷积核一样大的小块
→ 从第i行到第i+h行，从第j列到第j+w列

举例：i=0, j=0, h=2, w=2
    X[0:2, 0:2] 就是左上角的2×2小块
```

```
X[i:i+h, j:j+w] * K
→ 小块和卷积核对应位置相乘（不是矩阵乘法！是逐个元素相乘）

小块：[0, 1]    卷积核：[0, 1]    相乘：[0×0, 1×1]   [0, 1]
      [3, 4]            [2, 3]          [3×2, 4×3] = [6, 12]
```

```
.sum()
→ 把所有结果加起来
→ 0 + 1 + 6 + 12 = 19

Y[0, 0] = 19
```

---

### 验证一下

```python
X = torch.tensor([[0.0, 1, 2],
                   [3, 4, 5],
                   [6, 7, 8]])

K = torch.tensor([[0.0, 1],
                   [2, 3]])

print(corr2d(X, K))
# 输出：tensor([[19., 25.],
#               [37., 43.]])
```

**和我们手算的结果一样！**

---

## 二、用这个运算构建卷积层

### 代码

```python
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias
```

### 逐行翻译

```python
class Conv2D(nn.Module):
```

```
定义一个卷积层，名字叫Conv2D
继承nn.Module（和之前学的一样）
```

---

```python
def __init__(self, kernel_size):
```

```
准备工作：需要告诉我卷积核多大

kernel_size就是核的大小
比如传入 (2, 2) 就是2×2的核
```

---

```python
self.weight = nn.Parameter(torch.rand(kernel_size))
```

```
torch.rand(kernel_size)  → 生成一个随机矩阵（比如2×2的随机数）
nn.Parameter(...)        → 告诉PyTorch：这是可学习的参数！要训练它！
self.weight = ...        → 绑在自己身上，叫weight

这就是卷积核！初始值是随机的，训练会调整它
```

---

```python
self.bias = nn.Parameter(torch.zeros(1))
```

```
偏置，初始为0
也是可学习的参数
就是一个数字，加到输出上
```

---

```python
def forward(self, X):
    return corr2d(X, self.weight) + self.bias
```

```
数据怎么走：
    把输入X和自己的卷积核(self.weight)做互相关运算
    再加上偏置(self.bias)
    返回结果
```

---

## 三、边缘检测的例子

### 3.1 构造一个简单的输入

```python
X = torch.ones(6, 8)       # 6×8的全1矩阵
X[:, 2:6] = 0              # 中间4列设为0
print(X)
```

```
输出：
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1

左边两列是1，中间四列是0，右边两列是1
在第2列和第3列之间有一条"边缘"（从白变黑）
在第5列和第6列之间有一条"边缘"（从黑变白）
```

### 3.2 构造一个边缘检测核

```python
K = torch.tensor([[1.0, -1.0]])    # 1×2的核
```

**这个核为什么能检测边缘？**

```
核是 [1, -1]

当窗口盖住两个相同的像素时：
    [1, 1] × [1, -1] = 1×1 + 1×(-1) = 0    ← 没有边缘
    [0, 0] × [1, -1] = 0×1 + 0×(-1) = 0    ← 没有边缘

当窗口盖住两个不同的像素时：
    [1, 0] × [1, -1] = 1×1 + 0×(-1) = 1    ← 检测到边缘！
    [0, 1] × [1, -1] = 0×1 + 1×(-1) = -1   ← 检测到边缘！

相同 → 输出0（没边缘）
不同 → 输出非0（有边缘！）
```

### 3.3 做卷积

```python
Y = corr2d(X, K)
print(Y)
```

```
输出：
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0

1的那一列 = 从白变黑的边缘
-1的那一列 = 从黑变白的边缘
0 = 没有边缘
```

**卷积成功检测出了竖直方向的边缘！**

---

## 四、用梯度下降学习卷积核

### 场景

```
现在假装你不知道K是[1, -1]
你只知道输入X和输出Y
问：能不能通过训练学出K？
```

### 代码

```python
# 用PyTorch自带的Conv2d
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
```

**参数解释：**

```
nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
          ^  ^              ^^^^^        ^^^^^
          |  |              核大小       不要偏置
          |  输出通道数=1
          输入通道数=1

现在只有1个输入1个输出（黑白图片）
核大小1×2（和我们手动构造的一样）
```

---

```python
# 把X和Y变成4维（PyTorch要求的格式）
X = X.reshape((1, 1, 6, 8))    # 1个样本，1个通道，6行8列
Y = Y.reshape((1, 1, 6, 7))    # 输出也要reshape
```

**为什么要4维？**

```
PyTorch的Conv2d要求输入必须是4维：
(批量大小, 通道数, 高, 宽)

我们就1张图，1个通道
所以前面加两个1
```

---

```python
# 训练10轮
for i in range(10):
    Y_hat = conv2d(X)                    # 用当前的核算输出
    loss = (Y_hat - Y) ** 2              # 算误差
    conv2d.zero_grad()                   # 清零梯度
    loss.sum().backward()                # 反向传播
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad   # 更新权重
    if (i + 1) % 2 == 0:
        print(f'第{i+1}轮, loss={loss.sum():.3f}')
```

### 逐行翻译

```python
Y_hat = conv2d(X)
```

```
把X丢进卷积层，用当前的（随机的）核算出结果
Y_hat是预测值
```

---

```python
loss = (Y_hat - Y) ** 2
```

```
预测值 - 真实值，然后平方
平方是为了让误差都变正数
loss越小说明核越接近正确答案
```

---

```python
conv2d.zero_grad()
```

```
清零之前的梯度
每次算新梯度之前必须清零（PyTorch的规矩）
```

---

```python
loss.sum().backward()
```

```
.sum() → 把所有误差加起来变成一个数
.backward() → 反向传播，算出梯度
```

---

```python
conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
```

```
手动更新权重（梯度下降）

conv2d.weight.data  → 当前的核的值
conv2d.weight.grad  → 核的梯度（该往哪个方向调）
3e-2 = 0.03         → 学习率（每次调多少）

新权重 = 旧权重 - 学习率 × 梯度
```

### 最终结果

```python
print(conv2d.weight.data.reshape((1, 2)))
# 输出：tensor([[ 0.99, -0.99]])
```

**学出来的核接近 [1, -1]，和我们手动构造的几乎一样！**

```
我们构造的：   [1.0,  -1.0]
学出来的：     [0.99, -0.99]

非常接近！说明网络通过训练自己学会了边缘检测
```

---

## 五、总结

### 你需要记住的

```
1. 卷积运算 = 小窗口滑动，对应位置相乘再求和
2. 输出大小 = 输入大小 - 核大小 + 1
3. 卷积核是可学习的参数，不需要手动设计
4. PyTorch的Conv2d输入必须是4维：(批量, 通道, 高, 宽)
```

### 使用PyTorch自带的Conv2d

```python
# 实际中直接用这个就行，不需要自己写corr2d
conv_layer = nn.Conv2d(
    in_channels=1,        # 输入通道数
    out_channels=1,       # 输出通道数
    kernel_size=(3, 3),   # 卷积核大小
    bias=False            # 要不要偏置
)
```